In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("datasets/train.csv.zip", compression="zip", header=0)
test = pd.read_csv("datasets/test.csv.zip", compression="zip", header=0)

In [3]:
images = train.values[:,1:].reshape(-1, 28, 28)[6]

In [4]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [5]:
inputs = tf.keras.Input(shape = (28, 28, 1))

C1 = tf.keras.layers.RandomContrast(0.2)(inputs)

M_1 = tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape = (28, 28, 1), padding="same")(C1)
M_1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=1, padding="same")(M_1)

M_2 = tf.keras.layers.Conv2D(8, 3, activation='relu', padding="same")(M_1)
M_2 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2, padding="same")(M_2)

M_3 = tf.keras.layers.Conv2D(4, 3, activation='relu', padding="same")(M_2)
M_3 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2, padding="valid")(M_3)

M = tf.keras.layers.Flatten()(M_3)
D1 = tf.keras.layers.Dense(16, activation="relu")(M)
D2 = tf.keras.layers.Dense(16, activation="relu")(D1)  

outputs = tf.keras.layers.Dense(10, activation="softmax", name="output")(D2)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']
)

In [6]:
for i in range(10):
    train[f'label_{i}'] = (train['label'] == i) * 1.0

In [7]:
X = train[[c for c in train.columns if 'pixel' in c]].values.reshape(len(train), 28, 28, 1)
y = train[[c for c in train.columns if 'label_' in c]]

In [8]:
model.fit(X, y, epochs = 20, validation_split = 0.2)

Epoch 1/20
1050/1050 [==============================] - 26s 24ms/step - loss: 1.1043 - accuracy: 0.6745 - val_loss: 0.3836 - val_accuracy: 0.8995
Epoch 2/20
1050/1050 [==============================] - 26s 25ms/step - loss: 0.2886 - accuracy: 0.9188 - val_loss: 0.2114 - val_accuracy: 0.9414
Epoch 3/20
1050/1050 [==============================] - 25s 24ms/step - loss: 0.2033 - accuracy: 0.9413 - val_loss: 0.1733 - val_accuracy: 0.9481
Epoch 4/20
1050/1050 [==============================] - 26s 24ms/step - loss: 0.1578 - accuracy: 0.9536 - val_loss: 0.1325 - val_accuracy: 0.9625
Epoch 5/20
1050/1050 [==============================] - 27s 26ms/step - loss: 0.1276 - accuracy: 0.9621 - val_loss: 0.1405 - val_accuracy: 0.9590
Epoch 6/20
1050/1050 [==============================] - 25s 23ms/step - loss: 0.1095 - accuracy: 0.9680 - val_loss: 0.1051 - val_accuracy: 0.9682
Epoch 7/20
1050/1050 [==============================] - 24s 22ms/step - loss: 0.0976 - accuracy: 0.9708 - val_loss: 0.1100 -

In [9]:
model.summary()

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 random_contrast (RandomCont  (None, 28, 28, 1)        0         
 rast)                                                           
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 28, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 8)         1160      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 8)        

In [11]:
test['Label'] = model.predict(
    test[[c for c in test.columns if 'pixel' in c]].values.reshape(len(test), 28, 28, 1)
).argmax(axis = 1)

In [13]:
test['ImageId'] = list(range(1, len(test) + 1))
test.to_csv('datasets/submission.csv', index=False, columns = ['ImageId', 'Label'])